## Inserting Similarity Matrix Mongodb

In [1]:
from pymongo import MongoClient 
import pandas as pd
import sys
import numpy as np

In [2]:

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


In [3]:
try: 
    conn = MongoClient('192.168.1.72', 27017) 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 


Connected successfully!!!


## Reading from mongodb

In [4]:
mydb = conn.SK


In [5]:
collection1 = mydb.OrderMaster


In [6]:
df1 = pd.DataFrame(list(collection1.find({},{"Skcode":1,"orderDetails":1,"_id":0})))

In [7]:
df1

,Skcode,orderDetails
0,SK8393,[{'GUID': f7667b25-5aea-0e48-8c99-f7105851cd50...
1,SK10454,[{'GUID': d6f02625-f65d-eb47-807b-5f6a37984c4f...
2,SK4160,[{'GUID': 5d89d8c5-985d-0441-80ec-f4569d641d8c...
3,SK9511,[{'GUID': 8257ebaa-8d5f-d74c-9bae-9d4842d08cbe...
4,SK8599,[{'GUID': 50bf8fda-7076-a849-96b6-034e024deb4c...
...,...,...
286806,SK7671,[{'GUID': a0f8d725-7cea-a44d-b7f8-76259164e87e...
286807,SK69885,[{'GUID': e5b17237-2c04-a44e-bcc5-01e229d99dc7...
286808,SK32903,[{'GUID': 6a880dd0-ed30-d847-86a7-e5aeabb89ca4...
286809,SK49952,[{'GUID': 930c7728-dfcd-4f49-959e-721c375a8c83...


## Data munging and cleaning

In [8]:
df1["item"] = [[i[j]['ItemId'] for j in range(len(i))] for i in df1['orderDetails']]

In [9]:
df1 = df1[['Skcode','item']]

In [10]:
df2 = df1.explode('item')

In [11]:
df2

,Skcode,item
0,SK8393,1576
0,SK8393,923
0,SK8393,933
0,SK8393,1197
0,SK8393,1784
...,...,...
286808,SK32903,209671
286809,SK49952,141822
286809,SK49952,136169
286810,SK45319,76655


In [12]:
df2['rating'] = df2.apply(lambda _: '1', axis=1)


In [13]:
df2.columns = ['userID','itemID', 'rating']

In [14]:
df2

,userID,itemID,rating
0,SK8393,1576,1
0,SK8393,923,1
0,SK8393,933,1
0,SK8393,1197,1
0,SK8393,1784,1
...,...,...,...
286808,SK32903,209671,1
286809,SK49952,141822,1
286809,SK49952,136169,1
286810,SK45319,76655,1


In [15]:
df2[df2.itemID == 319466]

,userID,itemID,rating
193074,SK53703,319466,1
216165,SK19689,319466,1
216229,SK19799,319466,1
241901,SK54635,319466,1
250274,SK64635,319466,1
268075,SK12079,319466,1


In [16]:
df2.itemID.nunique()

15828

In [17]:
# df2[df2.duplicated()]

In [18]:
df3 = df2.drop_duplicates()


In [19]:
df3

,userID,itemID,rating
0,SK8393,1576,1
0,SK8393,923,1
0,SK8393,933,1
0,SK8393,1197,1
0,SK8393,1784,1
...,...,...,...
286806,SK7671,165,1
286807,SK69885,177138,1
286807,SK69885,266027,1
286810,SK45319,76655,1


In [22]:
df3.to_excel("Data_to_fit_in_surprise.xlsx")  

In [94]:
df2['rating'] = df2['rating'].astype(int)


In [95]:
#df2[df2.itemID == 3]

In [96]:
a=df2.groupby('itemID')['rating'].apply(list)
b = a.to_frame()
c = b.copy()

In [97]:
c

,rating
itemID,
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...
319616,[1]
320773,"[1, 1]"
320775,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


## Calculating Frequency

In [98]:
c['frequency']  = [sum(i) for i in c['rating']]

In [99]:
c

,rating,frequency
itemID,,
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",15
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",49
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",66
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",63
...,...,...
319616,[1],1
320773,"[1, 1]",2
320775,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10


In [100]:
c1 = c.reset_index()


In [101]:
c1

,itemID,rating,frequency
0,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",15
1,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123
2,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",49
3,3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",66
4,4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",63
...,...,...,...
14235,319616,[1],1
14236,320773,"[1, 1]",2
14237,320775,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10
14238,321387,"[1, 1]",2


In [102]:
c1.index.name = 'indexing'

In [167]:
c1

,itemID,rating,frequency
indexing,,,
0,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",15
1,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123
2,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",49
3,3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",66
4,4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",63
...,...,...,...
14235,319616,[1],1
14236,320773,"[1, 1]",2
14237,320775,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10


In [184]:
mapped = zip([i for i in c1['itemID']] , [j for j in c1['frequency']])

In [196]:
freq = [j for j in c1['frequency']]

In [199]:
freq.append(0)

In [185]:
lfreq = list(mapped)

In [186]:
#l

In [187]:
lfreq.append((322758,0))

## Calculating Cosine Similarity Matrix using surprise

In [217]:
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

In [218]:
#testSubject = 'SK4005'


temp = []

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df3[['userID', 'itemID', 'rating']], reader)



k = 10
# Load our data set and compute the user similarity matrix
#ml = MovieLens()
#data = ml.loadMovieLensLatestSmall()

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': False
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()



Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [287]:
simsMatrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [288]:
simsMatrix.shape

(14241, 14241)

In [290]:
simsMatrix = simsMatrix * freq

In [291]:
l = [i for i in c1['itemID'] ]


In [292]:
l.append(322758)

In [293]:
len(l)

14241

In [295]:
simsMatrix

array([[ 15., 123.,  49., ...,   0.,   0.,   0.],
       [ 15., 123.,  49., ...,   0.,   0.,   0.],
       [ 15., 123.,  49., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   2.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,  11.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [297]:
df = pd.DataFrame(simsMatrix , columns = l, index = l)

In [298]:
df

,0,1,2,3,4,5,6,7,8,9,...,319316,319410,319411,319466,319616,320773,320775,321387,321577,322758
0,15.0,123.0,49.0,66.0,63.0,4.0,107.0,54.0,60.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,123.0,49.0,66.0,63.0,4.0,107.0,54.0,60.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.0,123.0,49.0,66.0,63.0,4.0,107.0,54.0,60.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.0,123.0,49.0,66.0,63.0,4.0,107.0,54.0,60.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.0,123.0,49.0,66.0,63.0,4.0,107.0,54.0,60.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,10.0,0.0,0.0,0.0
320775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,10.0,0.0,0.0,0.0
321387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
321577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0


In [299]:
df.values

array([[ 15., 123.,  49., ...,   0.,   0.,   0.],
       [ 15., 123.,  49., ...,   0.,   0.,   0.],
       [ 15., 123.,  49., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   2.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,  11.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [300]:
lcolumn = list(df.columns)

In [301]:
len(lcolumn)

14241

In [302]:
df[319466]

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
320773    0.0
320775    0.0
321387    0.0
321577    0.0
322758    0.0
Name: 319466, Length: 14241, dtype: float64

In [303]:
df.values.shape

(14241, 14241)

In [314]:
itemid = 2


In [315]:
mapped = zip(lcolumn , df[itemID])

In [316]:
lis = list(mapped)

In [317]:
#lis

In [318]:
kNeighbors = [(itemid,1.0)]

In [320]:
# Get similar items to stuff we liked (weighted by rating)
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = df[itemID].values
    mapped = zip(lcolumn , df[itemID])
    lis = list(mapped)
    for innerID, score in lis:
        candidates[innerID] += score * (rating )
#print(candidates)
    
# Build a dictionary of stuff the user has already seen
#watched = {}
#for itemID, rating in trainSet.ur[testUserInnerID]:
#    watched[itemID] = 1


pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    print(itemID,ratingSum)
    pos += 1
    if (pos > 10):
        break



0 49.0
1 49.0
2 49.0
3 49.0
4 49.0
5 49.0
6 49.0
7 49.0
8 49.0
9 49.0
10 49.0


In [35]:
# Get top N similar users to our test subject
# (Alternate approach would be to select users up to some similarity threshold - try it!)
testUserInnerID = trainSet.to_inner_uid(testSubject)
similarityRow = simsMatrix[testUserInnerID]

similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append( (innerID, score) )

kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

# for tuning  collabrative filtering algorithms

#
#kNeighbors = []

#for rating in testUserRatings:     #by increasing threshold newer items will return to result 
#    if rating[1] > 0.2:
#        kNeighbors.append(rating)



# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore

# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        # movieID = trainSet.to_raw_iid(itemID)
        # print(ml.getMovieName(int(movieID)), ratingSum)
        print(itemID)
        #temp.append(itemID) 
        pos += 1
        if (pos > 10):
            break

924
1168
1093
470
1540
992
1030
1293
111
329
1262


In [36]:
kNeighbors

[(0, 1.0),
 (1, 1.0),
 (2, 1.0),
 (6, 1.0),
 (8, 1.0),
 (9, 1.0),
 (10, 1.0),
 (11, 1.0),
 (12, 1.0),
 (13, 1.0)]

In [37]:
d = {}

## Inserting the matrix in database 

In [35]:
mydb = conn.ankit_database


In [36]:
collection2 = mydb.RecSys


In [42]:
collection2.insert(d)


C:\Users\Ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5e00d9a5f920838303842b00')

In [43]:
# fibonnaci series